## 数据填充

**实验任务：**填充泰坦尼克号乘客数据集的缺失值

**实验课时：**0.5课时

**实验目的：**

* 了解数据填充在机器学习中的作用；
* 理解填充训练集和测试集的差异；
* 掌握数据填充的实现方法。

在前几节中，可以看出变量`age`、`fare`和`body`存在缺失值。有些模型可以接受有缺失值的数据样本，而有些模型要求数据样本完全没有缺失值。

载入需要用到的程序包。

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Imputer as SimpleImputer
pd.set_option('mode.chained_assignment',None)

读取读取泰坦尼克号乘客数据集。

In [6]:
titanic3_file_path="./titanic3.xls"
titanic3 = pd.read_excel(titanic3_file_path)
titanic3.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


随机划分训练集和测试集。

In [16]:
X = titanic3[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]
y = titanic3['survived']
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 123)

调用数据框的函数`isnull()`得到训练集和测试集中包含缺失值的行索引。

In [17]:
train_na_idx = train_X.isnull().any(axis = 1)
test_na_idx = test_X.isnull().any(axis = 1)
train_X.loc[train_na_idx].head()

,pclass,sex,age,sibsp,parch,fare,embarked
235,1,male,NaN,0,0,39.6000,S
973,3,male,NaN,0,0,7.5500,S
524,2,male,NaN,0,0,13.8625,C
37,1,male,NaN,0,0,26.5500,S
928,3,female,NaN,1,0,14.4542,C


### 1. 填充训练集

调用程序包`sklearn.impute`中的构造函数`SimpleImputer()`创建填充器，该函数主要是对`numpy`浮点数数组进行操作。其中

* 第1个参数`missing_values`表示缺失值的表示方式，默认为`'NaN'`，即表示为`np.nan`；
* 第2个参数`strategy`表示填充策略，`'mean'`为用均值填充，`'median'`为用中位数填充，`'most_frequent'`为用众数填充，默认为`'mean'`；
* 第3个参数`axis`表示填充轴，0为按列填充，1为按行填充；
* 返回结果为一个填充器对象。

以下例子用**中位数**填充所有**训练集**中的数值变量，即变量`age`（列索引为2）和`fare`（列索引为5）。

调用填充器的函数`fit()`训练填充器。

In [18]:
imp = SimpleImputer(strategy='median')
imp.fit(train_X[['age','fare']])

Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

调用填充器的函数`transform()`填充训练集，返回填充完成的Numpy数组。查看填充器的属性`statistics_`得到每个变量的填充值。

In [19]:
train_X[['age','fare']] = imp.transform(train_X[['age','fare']])
imp.statistics_

array([27.    , 13.8625])

调用数据框的函数`median()`得到变量的中位数，与填充器得到的填充值完全一致。

In [20]:
train_X[['age','fare']].median()

age     27.0000
fare    13.8625
dtype: float64

查看在原数据集中包含缺失值的记录填充后的结果。

In [21]:
train_X[train_na_idx][:10]

,pclass,sex,age,sibsp,parch,fare,embarked
235,1,male,27.0,0,0,39.6000,S
973,3,male,27.0,0,0,7.5500,S
524,2,male,27.0,0,0,13.8625,C
37,1,male,27.0,0,0,26.5500,S
928,3,female,27.0,1,0,14.4542,C
176,1,female,27.0,1,0,51.8625,S
1004,3,female,27.0,0,0,7.7875,Q
40,1,male,27.0,0,0,39.6000,C
1225,3,male,60.5,0,0,13.8625,S
680,3,male,27.0,0,0,7.2250,C


可以看出，训练集中的变量`age`都填充成了27，变量`fare`填充成了13.86。

### 2. 填充测试集

做数据填充时，无论是用统计量还是机器学习模型，都必须用**训练集**中的信息。在做模型测试前对测试集做数据填充时，也需要用训练集的统计量或基于训练集训练的机器学习模型。

调用填充器的函数`transform()`填充测试集。

In [22]:
test_X[['age','fare']] = imp.transform(test_X[['age','fare']])
test_X[test_na_idx][:10]

,pclass,sex,age,sibsp,parch,fare,embarked
798,3,male,27.0,0,0,7.0500,S
1009,3,male,27.0,0,0,7.7500,Q
1078,3,female,27.0,0,0,7.8792,Q
384,2,male,27.0,0,0,0.0000,S
872,3,female,27.0,0,0,8.0500,S
204,1,female,27.0,1,0,82.1708,C
1184,3,male,27.0,2,0,21.6792,C
902,3,male,27.0,1,2,23.4500,S
1166,3,male,27.0,0,0,7.2250,C
946,3,male,27.0,0,0,56.4958,S


可以看出，测试集中的变量`age`也都填充成了27。
